<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Interactive-Visualization-Lab" data-toc-modified-id="Interactive-Visualization-Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Interactive Visualization Lab</a></span><ul class="toc-item"><li><span><a href="#1.-Create-an-interactive-bar-chart-showing-total-quantity-and-revenue-by-country-(excluding-United-Kingdom)-for-the-month-of-April-2011." data-toc-modified-id="1.-Create-an-interactive-bar-chart-showing-total-quantity-and-revenue-by-country-(excluding-United-Kingdom)-for-the-month-of-April-2011.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.</a></span></li><li><span><a href="#2.-Create-an-interactive-line-chart-showing-quantity-and-revenue-sold-to-France-between-January-1st-and-May-31st-2011." data-toc-modified-id="2.-Create-an-interactive-line-chart-showing-quantity-and-revenue-sold-to-France-between-January-1st-and-May-31st-2011.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.</a></span></li><li><span><a href="#3.-Create-an-interactive-scatter-plot-showing-the-relationship-between-average-quantity-(x-axis)-and-average-unit-price-(y-axis)-for-the-product-PARTY-BUNTING-with-the-plot-points-color-coded-by-country-(categories)." data-toc-modified-id="3.-Create-an-interactive-scatter-plot-showing-the-relationship-between-average-quantity-(x-axis)-and-average-unit-price-(y-axis)-for-the-product-PARTY-BUNTING-with-the-plot-points-color-coded-by-country-(categories).-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).</a></span></li><li><span><a href="#4.-Create-a-set-of-interactive-histograms-showing-the-distributions-of-quantity-per-invoice-for-the-following-countries:-EIRE,-Germany,-France,-and-Netherlands." data-toc-modified-id="4.-Create-a-set-of-interactive-histograms-showing-the-distributions-of-quantity-per-invoice-for-the-following-countries:-EIRE,-Germany,-France,-and-Netherlands.-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.</a></span></li><li><span><a href="#5.-Create-an-interactive-side-by-side-bar-chart-showing-the-revenue-by-country-listed-below-(bars)-for-each-of-the-products-listed-below." data-toc-modified-id="5.-Create-an-interactive-side-by-side-bar-chart-showing-the-revenue-by-country-listed-below-(bars)-for-each-of-the-products-listed-below.-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.</a></span></li><li><span><a href="#6.-Create-an-interactive-line-chart-showing-quantity-sold-by-day-for-the-United-Kingdom.-Add-drop-down-boxes-for-Year-and-Month-that-allow-you-to-filter-the-date-range-that-appears-in-the-chart." data-toc-modified-id="6.-Create-an-interactive-line-chart-showing-quantity-sold-by-day-for-the-United-Kingdom.-Add-drop-down-boxes-for-Year-and-Month-that-allow-you-to-filter-the-date-range-that-appears-in-the-chart.-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.</a></span></li><li><span><a href="#7.-Create-an-interactive-scatter-plot-that-plots-number-of-invoices-(x-axis)-vs.-number-of-customers-(y-axis)-and-the-plot-points-represent-individual-products.-Add-two-sliders-that-control-the-x-and-y-axis-ranges." data-toc-modified-id="7.-Create-an-interactive-scatter-plot-that-plots-number-of-invoices-(x-axis)-vs.-number-of-customers-(y-axis)-and-the-plot-points-represent-individual-products.-Add-two-sliders-that-control-the-x-and-y-axis-ranges.-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.</a></span></li><li><span><a href="#8.-Creat-an-interactive-bar-chart-that-shows-revenue-by-product-description.-Add-a-text-field-widget-that-filters-the-results-to-show-the-product-that-contain-the-text-entered-in-their-description." data-toc-modified-id="8.-Creat-an-interactive-bar-chart-that-shows-revenue-by-product-description.-Add-a-text-field-widget-that-filters-the-results-to-show-the-product-that-contain-the-text-entered-in-their-description.-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.</a></span></li></ul></li></ul></div>

# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [39]:
import pandas as pd
import plotly.express as px
import cufflinks as cf
from ipywidgets import interact
from datetime import date as dt
import seaborn as sns

cf.go_offline()

In [2]:
data = pd.read_csv('../data/Online Retail.csv')

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [3]:
# Check data types
# Invoice date is set as string

data.dtypes

InvoiceNo        int64
InvoiceDate     object
StockCode       object
Description     object
Quantity         int64
UnitPrice      float64
Revenue        float64
CustomerID       int64
Country         object
dtype: object

In [4]:
# Cast InvoiceDate to datetime 

data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [5]:
# Create a DataFrame with the information filtered: April 2011, Country different to UK

data1 = data[(data['InvoiceDate'].dt.month == 4)&(data['InvoiceDate'].dt.year == 2011)&(data['Country'] != 'United Kingdom')].groupby('Country')['Quantity', 'Revenue'].sum()

<ipython-input-5-d07be013af50>:3: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [6]:
# Cufflinks interactive bar plot

data1.iplot(kind='bar')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [7]:
# Create a DataFrame with the information filtered: From January to May 2011, Country is France, grouped by InvoiceDate

data2 = data[(data['InvoiceDate']< '2011-05-31')&(data['InvoiceDate']> '2011-01-01')&(data['Country'] =='France')].groupby('InvoiceDate')['Quantity', 'Revenue', 'InvoiceDate'].sum()

<ipython-input-7-a1a7709f820c>:3: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [8]:
# Plotly Express interactive line plot

px.line(data2)

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [17]:
# Create a DataFrame with the information filtered: Description is PARTY BUNTING, grouped by Country

data3 = data[data['Description']=='PARTY BUNTING'].groupby('Country')['Quantity', 'UnitPrice'].mean().reset_index()

<ipython-input-17-e368766fbc98>:3: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [18]:
# Cufflinks interactive scatte plot

data3.iplot(kind='scatter', x='Quantity', y='UnitPrice', mode='markers', categories='Country')

C:\Users\carlo\anaconda3\lib\site-packages\cufflinks\plotlytools.py:807: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

C:\Users\carlo\anaconda3\lib\site-packages\cufflinks\plotlytools.py:810: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [28]:
# Create a list with the countries

countries = ['EIRE', 'Germany', 'France', 'Netherlands']

# For loop per country to create a histogram with quantities per invoice distribution
for country in countries:
    data[data['Country'] ==country].groupby(['InvoiceNo'])['Quantity'].sum().iplot(kind ='hist', title =str(country))

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [20]:
# Lists of products and countries

product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']


In [67]:
# Check the products in the list
# Check the countries in the list
# Group by Country and desciption with sum as aggregation function

data5 = data[data['Description'].isin(product_list)&data['Country'].isin(country_list)][['Country','Description','Revenue']].groupby(['Country','Description']).sum().reset_index()

In [68]:
# Plotly express interactive scatter plot

px.bar(data5, x='Description', y='Revenue', color='Country', barmode='group')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [35]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [70]:
# Introduce interactive filters with interact
# Inside the function, create a new dataset with filtered values depending on interact
# Seaborn line plot

@interact(month=uk['Month'].sort_values().unique(), year=uk['Year'].sort_values().unique())
def explore(month, year):
    data6 = uk[(uk['Month'] ==month) & (uk['Year'] == year)]
    sns.relplot(data=data6, x='Day', y='Quantity',kind='line')

interactive(children=(Dropdown(description='month', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=1),…

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [58]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

In [71]:
# Introduce interactive sliders with interact for invoices and customers
# Cufflinks line plot

@interact(invoices=(0, products['InvoiceNo'].max()), customers=(0, products['CustomerID'].max()))
def scatter_plot(invoices, customers):
    products.iplot(kind='scatter',x='InvoiceNo', y='CustomerID', mode='markers')

interactive(children=(IntSlider(value=945, description='invoices', max=1891), IntSlider(value=410, description…

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [80]:
# Introduce interactive text filter with interact for products
# Create new DataFrame grouping by description and summing the revenue
# Cufflinks bar plot

@interact(Product='')
def bar(Product):
    Product = Product.upper()
    data8 = uk.groupby('Description')[['Revenue']].sum().reset_index()
    data8[data8['Description'].str.contains(Product)].iplot(kind='bar', x='Description', y='Revenue')

interactive(children=(Text(value='', description='Product'), Output()), _dom_classes=('widget-interact',))